In [ ]:
Skip to content
You said:
import pika
url = ['z','e','f']

# Conecta no RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')


for t in url:
    # Envia uma mensagem para pedidos novos
    channel.basic_publish(
        exchange='consult_exchange',
        routing_key='links.novo',
        body='Pedido 123 criado'
    )

# Envia uma mensagem para pedidos cancelados
# channel.basic_publish(
#     exchange='pedidos_ex',
#    routing_key='pedido.cancelado',
#    body='Pedido 456 cancelado'
#)

print("Mensagens enviadas!")
connection.close()

pq nao tem queue
ChatGPT said:

O seu código declara uma exchange (consult_exchange), envia mensagens para ela, mas não declara nenhuma queue nem faz o binding entre uma queue e a exchange. Por isso, as mensagens são perdidas, a menos que já exista alguma fila previamente configurada no RabbitMQ e ligada à exchange.
Por que as mensagens "somem"?

No RabbitMQ, o fluxo básico de mensagens é:

Publisher → Exchange → Queue(s) → Consumer(s)

Se nenhuma queue estiver ligada (bound) à exchange com a routing key correspondente, as mensagens:

    São descartadas automaticamente (não ficam armazenadas em lugar nenhum);

    E não causam erro no basic_publish() – ou seja, o seu código continua funcionando "normalmente", mesmo sem fazer nada útil.

Como corrigir

Você precisa:

    Declarar uma fila (queue).

    Fazer o binding dessa fila com a exchange usando a mesma routing_key que você usa para publicar.

Código corrigido com fila

import pika

url = ['z', 'e', 'f']

# Conecta ao RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')

# Declara a fila
channel.queue_declare(queue='consult_queue')

# Faz o binding da fila à exchange com a routing key usada
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Envia mensagens
for t in url:
    channel.basic_publish(
        exchange='consult_exchange',
        routing_key='links.novo',
        body=f'Pedido 123 criado: {t}'
    )

print("Mensagens enviadas!")
connection.close()

Dica extra: Debug – como saber se a fila recebeu?

Você pode usar o painel de administração do RabbitMQ (geralmente acessível em http://localhost:15672) para ver:

    Se a queue foi criada;

    Se ela tem mensagens prontas para serem consumidas;

    Se a exchange e a queue estão ligadas corretamente.

Se quiser, posso te ajudar também a escrever o código do consumer que consome da fila consult_queue. Deseja isso?
You said:
exchange, queue,  routing_key como que funcionam
ChatGPT said:

Ótima pergunta! Entender bem como funcionam exchange, queue e routing_key é essencial para usar o RabbitMQ corretamente.

Vamos explicar de forma simples e objetiva, com analogias e exemplos práticos.
🧠 Conceitos Básicos
✅ exchange (troca, distribuidor de mensagens)

A exchange é responsável por receber mensagens de um produtor (publisher) e decidir para onde elas vão.

Ela não armazena mensagens – só as encaminha para as filas (queues).
✅ queue (fila)

A fila é onde a mensagem fica armazenada até ser consumida por um consumidor (consumer).

    Se ninguém estiver consumindo a mensagem, ela fica "parada" na fila esperando.

✅ routing_key (chave de roteamento)

É uma etiqueta que acompanha a mensagem e ajuda a exchange a decidir para qual fila a mensagem vai (baseado nas regras de binding entre exchange e fila).
🧭 Como tudo se conecta?

Aqui está um resumo visual e conceitual:

[ Producer ]
     |
     |  (routing_key='user.created')
     v
[ Exchange ]  --------> [ Queue A ]
                 \
                  ---> [ Queue B ]

🧪 Tipos de Exchange

Existem 4 tipos principais de exchange no RabbitMQ:
Tipo de Exchange	O que faz	routing_key usada?	Exemplo
direct	Entrega para filas com routing_key exata	✅ Sim	Mensagem com key user.created vai para fila ligada com essa key
fanout	Entrega para todas as filas ligadas	❌ Ignorada	Broadcast (ex: enviar mensagem para todos)
topic	Entrega com base em padrões (wildcards)	✅ Sim + curingas	user.*, user.#
headers	Usa headers em vez de routing_key	❌ Usa headers	Avançado – raramente usado
📦 Exemplo prático com direct

Imagine um sistema de pedidos:

    Temos uma exchange chamada pedidos_ex

    Temos duas filas:

        fila_novos_pedidos

        fila_pedidos_cancelados

Fazemos o binding assim:

channel.queue_bind(exchange='pedidos_ex', queue='fila_novos_pedidos', routing_key='pedido.novo')
channel.queue_bind(exchange='pedidos_ex', queue='fila_pedidos_cancelados', routing_key='pedido.cancelado')

Agora, se você publicar:

channel.basic_publish(exchange='pedidos_ex', routing_key='pedido.novo', body='Pedido 123 criado')

➡️ A mensagem vai para fila_novos_pedidos.
🧠 Resumo rápido
Elemento	Responsável por...
exchange	Receber a mensagem e decidir para onde ela vai
queue	Guardar a mensagem até alguém consumir
routing_key	Definir como a mensagem é roteada para as filas

Se quiser, posso te mostrar um exemplo completo com produtor + consumidor, ou um gráfico explicando visualmente com setas. Deseja isso?
You said:
etiqueta ajuda? como assim
ChatGPT said:

Ótima observação. Vamos esclarecer isso.

Quando eu disse que a routing_key é como uma etiqueta, quis dizer o seguinte:
🎯 Analogia: Correios

Imagine que você está enviando uma carta.

    Você escreve o endereço do destinatário no envelope.

    O carteiro (🐰 exchange) lê o endereço e entrega a carta para a pessoa certa (fila).

👉 Neste caso:

    A carta é a mensagem (body)

    O endereço é a routing_key

    O carteiro é a exchange

    A casa do destinatário é a queue

Se o endereço estiver errado ou ninguém tiver uma casa com aquele nome/endereço, a carta não será entregue.
🧪 No RabbitMQ

A routing_key funciona da mesma forma:
é uma etiqueta (chave de roteamento) usada para decidir para qual fila a mensagem deve ir.
Exemplo prático:

Vamos dizer que temos essas filas ligadas a uma exchange do tipo direct:

channel.queue_bind(exchange='pedidos_ex', queue='fila_novos', routing_key='pedido.novo')
channel.queue_bind(exchange='pedidos_ex', queue='fila_cancelados', routing_key='pedido.cancelado')

Agora, se você envia:

channel.basic_publish(exchange='pedidos_ex', routing_key='pedido.novo', body='Novo pedido!')

A exchange vê a routing_key = 'pedido.novo' e pensa:

    “Ah! A fila fila_novos quer mensagens com essa etiqueta, vou mandar pra ela.”

Mas se você usar uma chave diferente, tipo:

routing_key = 'pedido.reembolsado'

E nenhuma fila estiver ligada com essa chave, a exchange vai descartar a mensagem.
✅ Moral da história:

A routing_key é uma forma de etiquetar a mensagem para que a exchange saiba para onde mandar.

Se ninguém estiver esperando por essa etiqueta (chave), a mensagem se perde.

Se quiser, posso montar um pequeno exemplo completo (com código) onde fica bem fácil visualizar tudo isso. Quer?
You said:
import pika

url = ['z', 'e', 'f']

# Conecta ao RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')

# Declara a fila
channel.queue_declare(queue='consult_queue')

# Faz o binding da fila à exchange com a routing key usada
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Envia mensagens
for t in url:
    print(t)
    channel.basic_publish(
        exchange='consult_exchange',
        routing_key='links.novo',
        body=f'Pedido 123 criado: {t}'
    )

print("Mensagens enviadas!")
connection.close()



import pika
import time

def callback(ch, method, properties, body):
    print("📥 [NOVO] Recebido:", body.decode())

    # Simula um processamento
    time.sleep(2)

    print("✅ [NOVO] Processado com sucesso.")
    
    # Envia ack manual
    ch.basic_ack(delivery_tag=method.delivery_tag)

# Conexão
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange e a fila
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')

# Liga a fila à exchange
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Limita a 1 mensagem por consumidor
channel.basic_qos(prefetch_count=1)

# Escuta a fila com ack manual
channel.basic_consume(
    queue='fila_novos_pedidos',
    on_message_callback=callback,
    auto_ack=False  # <- importante!
)

print('🟢 Aguardando novos pedidos...')
channel.start_consuming()
ChatGPT said:

Ótimo que você já montou o producer e o consumer com o RabbitMQ! 🚀
Mas tem um erro discreto no seu código que vai impedir o consumer de funcionar corretamente. Vamos revisar e corrigir juntos.
🧨 Problema principal

No seu consumer, você está escutando uma fila chamada:

queue='fila_novos_pedidos'

Porém, no producer, você está declarando e enviando mensagens para a fila chamada:

queue='consult_queue'

👉 Ou seja, o consumer está esperando em uma fila diferente daquela que o producer está usando.
✅ Solução

Troque o nome da fila no consumer para consult_queue, igual ao do producer.
✅ Código corrigido (consumer):

import pika
import time

def callback(ch, method, properties, body):
    print("📥 [NOVO] Recebido:", body.decode())

    # Simula um processamento
    time.sleep(2)

    print("✅ [NOVO] Processado com sucesso.")
    
    # Envia ack manual
    ch.basic_ack(delivery_tag=method.delivery_tag)

# Conexão
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange e a fila
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')

# Liga a fila à exchange
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Limita a 1 mensagem por consumidor
channel.basic_qos(prefetch_count=1)

# Escuta a fila com ack manual
channel.basic_consume(
    queue='consult_queue',  # CORRIGIDO AQUI!
    on_message_callback=callback,
    auto_ack=False
)

print('🟢 Aguardando novos pedidos...')
channel.start_consuming()

🧪 Agora o fluxo vai funcionar assim:

    O producer envia 3 mensagens com routing_key='links.novo' para a exchange='consult_exchange'.

    A exchange entrega as mensagens para a queue='consult_queue', que está ligada corretamente.

    O consumer consome essas mensagens, uma por vez, e imprime cada uma delas.

Se quiser, posso te ajudar a testar isso com múltiplos consumidores ou simular falhas. Está funcionando agora?
You said:
How to get message
ChatGPT said:

To get messages manually from the queue using the RabbitMQ Management Web UI, follow these steps based on your screenshot:
✅ How to Get (Read) Messages from the Queue
1. Go to the Queue Page

You're already there — the queue name is consult_queue.
2. Scroll Down to the “Get messages” Section

You will see this block (you already highlighted it):

    Get messages
    Warning: getting messages from a queue is a destructive action.
    [...]

3. Configure the Get Settings

Use the following values:

    Ack Mode:
    ✔ Ack message requeue false (or true if you want to leave the message in the queue)

    Encoding:
    ✔ Auto string / base64

    Messages:
    ✔ 1 (or however many messages you want to read)

4. Click “Get Message(s)”

After clicking it, a section just below will appear showing the message(s) retrieved, something like:

Payload: Pedido 123 criado: z

Or similar, depending on what you published.
📌 Notes

    This is a manual fetch, not consumption by a consumer.

    Once retrieved (with requeue false), the message is removed from the queue.

    If the queue is empty, it will simply say No messages.

🛠 Tip: Use It Only for Debug/Test

This tool is super useful for debugging or inspecting a few messages, but in production, messages should always be consumed by a consumer script (like the one you wrote with start_consuming()).

Would you like to test requeueing or simulate multiple pending messages too?
You said:
how it  would be
ChatGPT said:

Great — let's walk through how to use the “Get Message(s)” feature with different options, including requeueing, fetching multiple messages, and what each setting means in practice.
🔄 1. Requeue = true vs false
Option	What Happens
Ack message requeue false	Message is removed from the queue permanently
Ack message requeue true	Message is kept in the queue for other consumers
✅ When to use each:

    Use false if you're just trying to consume the message.

    Use true if you want to peek at the message without removing it.

📦 2. Number of Messages

You can choose how many messages to pull from the queue at once.

    Messages: 1 → Gets 1 message (default)

    Messages: 10 → Gets 10 messages (if available)

    ⚠️ These messages will all be removed if requeue = false.

🛠 Example Scenarios
✅ Scenario 1: Consume 1 message (like a real consumer)

    Ack Mode: Ack message requeue false

    Messages: 1

    ✅ This simulates a consumer pulling 1 message and removing it from the queue.

👁️ Scenario 2: Peek at message, leave in queue

    Ack Mode: Ack message requeue true

    Messages: 1

    ✅ Lets you see the message, but not delete it — good for debugging.

🧪 Scenario 3: Drain all messages for inspection

    Ack Mode: Ack message requeue false

    Messages: 10 (or higher if you know more messages exist)

    ✅ Use this to quickly clear the queue manually and inspect contents.

🧼 How to Clear a Queue Completely

If you want to clear all messages from a queue (not just get them):

    Go to the queue (consult_queue)

    Click on the "Purge" button (top-right area)

    Confirm

⚠️ This removes all messages immediately, with no chance to recover.
🔁 Want to simulate messages to test?

You can use the “Publish Message” section above to send test messages:

    Exchange: consult_exchange

    Routing key: links.novo

    Payload: Test message

Then use “Get Message(s)” to read it.

Would you like an example of publishing and retrieving a message step-by-step using the UI?
You said:
me ajuda a completar e conseguir get message
ChatGPT said:

Claro! Vamos passo a passo para que você consiga:

    Publicar uma mensagem corretamente na fila (consult_queue)

    E depois usar o botão “Get Message(s)” para recuperar essa mensagem

✅ ETAPA 1: Publicar uma mensagem corretamente

Siga isso na interface do RabbitMQ (Web UI):
Vá até a aba da fila consult_queue.
Role até a seção “Publish message” e preencha assim:

    Exchange:
    consult_exchange

    Routing key:
    links.novo

        Isso é essencial — tem que ser exatamente igual ao binding da fila.

    Payload:
    Coloque qualquer coisa, exemplo:
    Pedido de teste 001

    Headers e Properties:
    Pode deixar em branco.

    Delivery mode:
    Pode deixar como está (1 - Non-persistent)

✅ Clique em “Publish message”

Se tudo estiver certo, a mensagem será roteada pela exchange para a fila consult_queue.
✅ ETAPA 2: Recuperar (get) a mensagem da fila
Logo abaixo, vá para a seção “Get messages” e configure:

    Ack mode:
    Ack message requeue false ✅
    (isso consome a mensagem — ela será removida da fila)

    Encoding:
    Auto string / base64

    Messages:
    1

✅ Clique em “Get Message(s)”
🟢 Resultado Esperado:

    Um bloco com o conteúdo da mensagem vai aparecer abaixo.

    Exemplo de resposta:

Payload: Pedido de teste 001

Se aparecer isso: Queue is empty, pode ser que:

    A mensagem não foi enviada corretamente;

    A routing_key estava errada;

    Ou alguém (um consumer?) já consumiu a mensagem.

🧪 Dica de Teste Rápido

Repita o processo:

    Publica a mensagem novamente

    Em menos de 5 segundos, vá no “Get Message(s)” e clique

Se ainda assim não funcionar, me diga:

    O que você colocou nos campos da publicação

    O que aparece no resultado do Get

Posso te ajudar corrigindo em tempo real.
You said:
como seria para 2 workers
ChatGPT said:

Ótima pergunta! Vamos ver como rodar 2 workers (consumidores) para a mesma fila no RabbitMQ.

Isso é muito útil quando você quer dividir a carga de trabalho entre processos (por exemplo, para processamento paralelo de mensagens).
🧩 Situação: Você já tem isso

    Exchange: consult_exchange

    Fila: consult_queue

    Routing key: links.novo

    Mensagens estão sendo publicadas corretamente

Agora, queremos dois workers consumindo da mesma fila consult_queue.
✅ COMO FUNCIONA

Se dois consumidores estão escutando a mesma fila, o RabbitMQ faz o seguinte:

    Distribui as mensagens entre os dois

    Cada mensagem vai para apenas um worker

    Isso é chamado de load balancing automático

🛠️ PASSO A PASSO: Criar 2 Workers
Worker 1 (arquivo: worker1.py)

import pika
import time

def callback(ch, method, properties, body):
    print("👷‍♂️ [WORKER 1] Recebido:", body.decode())
    time.sleep(2)
    print("✅ [WORKER 1] Processado")
    ch.basic_ack(delivery_tag=method.delivery_tag)

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

channel.basic_qos(prefetch_count=1)  # Limita a 1 msg por vez
channel.basic_consume(queue='consult_queue', on_message_callback=callback, auto_ack=False)

print('🔄 [WORKER 1] Esperando mensagens...')
channel.start_consuming()

Worker 2 (arquivo: worker2.py)

import pika
import time

def callback(ch, method, properties, body):
    print("🧑‍🔧 [WORKER 2] Recebido:", body.decode())
    time.sleep(2)
    print("✅ [WORKER 2] Processado")
    ch.basic_ack(delivery_tag=method.delivery_tag)

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='consult_queue', on_message_callback=callback, auto_ack=False)

print('🔄 [WORKER 2] Esperando mensagens...')
channel.start_consuming()

▶️ COMO RODAR

    Abra dois terminais diferentes

    Execute:

python worker1.py

E no outro:

    python worker2.py

    Agora envie várias mensagens com seu producer.

🧪 Resultado Esperado:

Suponha que você envie 6 mensagens.

Você deve ver algo como:
No terminal do Worker 1:

👷‍♂️ [WORKER 1] Recebido: Pedido 123 criado: z
✅ [WORKER 1] Processado
👷‍♂️ [WORKER 1] Recebido: Pedido 123 criado: f
✅ [WORKER 1] Processado
👷‍♂️ [WORKER 1] Recebido: Pedido 123 criado: b
✅ [WORKER 1] Processado

No terminal do Worker 2:

🧑‍🔧 [WORKER 2] Recebido: Pedido 123 criado: e
✅ [WORKER 2] Processado
🧑‍🔧 [WORKER 2] Recebido: Pedido 123 criado: a
✅ [WORKER 2] Processado
🧑‍🔧 [WORKER 2] Recebido: Pedido 123 criado: c
✅ [WORKER 2] Processado

⚠️ A ordem pode variar, mas a ideia é: cada mensagem vai para só 1 worker.

Se você quiser que todos os workers recebam todas as mensagens, aí é outro cenário (usar fanout + uma fila por worker). Posso mostrar esse também, se for o caso.

Quer testar esse exemplo agora? Posso te ajudar em tempo real.
You said:
version: "3.2"
services:
  rabbitmq:
    image: rabbitmq:3-management-alpine
    container_name: 'rabbitmq'
    ports:
        - 5672:5672
        - 15672:15672
    volumes:
        - ~/.docker-conf/rabbitmq/data/:/var/lib/rabbitmq/
        - ~/.docker-conf/rabbitmq/log/:/var/log/rabbitmq
    networks:
        - rabbitmq_net

networks:
  rabbitmq_net:
    driver: bridge

como ficaria  o docker compose
ChatGPT said:

Seu docker-compose.yml já está muito bom e quase pronto para uso! 👏
Você já está usando a imagem certa (rabbitmq:3-management-alpine) com a interface web no localhost:15672.

Mas vou te mostrar uma versão revisada, organizada e comentada, com sugestões para facilitar o uso e evitar problemas.
✅ Versão Recomendada do docker-compose.yml

version: "3.8"

services:
  rabbitmq:
    image: rabbitmq:3-management-alpine
    container_name: rabbitmq
    restart: unless-stopped

    ports:
      - "5672:5672"     # Porta usada pelos clientes (ex: Python pika)
      - "15672:15672"   # Interface web de gerenciamento

    environment:
      RABBITMQ_DEFAULT_USER: admin      # Usuário padrão
      RABBITMQ_DEFAULT_PASS: admin123   # Senha padrão

    volumes:
      - ~/.docker-conf/rabbitmq/data:/var/lib/rabbitmq
      - ~/.docker-conf/rabbitmq/log:/var/log/rabbitmq

    networks:
      - rabbitmq_net

networks:
  rabbitmq_net:
    driver: bridge

✅ O que esse docker-compose.yml faz
Item	Explicação
rabbitmq:3-management-alpine	Imagem leve com painel web (Management Plugin)
5672:5672	Porta para conexões AMQP (ex: scripts Python usando pika)
15672:15672	Porta da interface web (http://localhost:15672)
RABBITMQ_DEFAULT_USER/PASS	Define o login inicial (evita acesso anônimo)
volumes	Persiste os dados da fila mesmo após reiniciar o container
networks	Isola a rede para usar com outros serviços (ex: seus workers)
🔐 Acessar a interface Web

Depois de rodar:

docker-compose up -d

Abra no navegador:

http://localhost:15672

Login:

    Usuário: admin

    Senha: admin123

🐍 Conectar no Python (pika)

Você pode usar este código:

import pika

credentials = pika.PlainCredentials('admin', 'admin123')
params = pika.ConnectionParameters('localhost', 5672, '/', credentials)

connection = pika.BlockingConnection(params)
channel = connection.channel()

⚠️ Se estiver no Windows

Troque ~/.docker-conf/... por um caminho compatível, como:

volumes:
  - ./data:/var/lib/rabbitmq
  - ./log:/var/log/rabbitmq

Se quiser, posso te ajudar a colocar os workers e o publisher no mesmo Docker Compose também, para tudo funcionar junto. Deseja isso?
You said:
no caso do worker nao precisaria mudar nada
ChatGPT said:

Boa pergunta! ✅

Se você está rodando o RabbitMQ via Docker e os workers estão fora do Docker (ou seja, rodando no seu sistema operacional local, via python worker.py), não precisa mudar nada no código dos workers, desde que eles se conectem a localhost.
🧩 Situação 1: Worker roda localmente (fora do Docker)

Se você rodar assim:

python worker1.py

E seu Docker está com as portas mapeadas assim:

ports:
  - "5672:5672"
  - "15672:15672"

Então, o código do worker funciona normalmente com:

pika.ConnectionParameters('localhost')

✅ Sem mudanças!
🧩 Situação 2: Worker também roda em um container

Se quiser colocar os workers dentro do Docker, aí sim você precisa mudar:

pika.ConnectionParameters('rabbitmq')  # nome do container, não 'localhost'

Porque dentro da rede Docker (rabbitmq_net), os serviços se enxergam pelo nome do container, e não por localhost.
🔄 Resumo rápido
Onde o Worker está rodando	Como conectar ao RabbitMQ
Fora do Docker	'localhost' (ou 127.0.0.1)
Dentro do Docker	'rabbitmq' (nome do serviço no compose)
🛠️ Quer colocar os workers no Docker também?

Se quiser, posso te mostrar como criar um docker-compose com:

    RabbitMQ

    Worker 1 (container Python)

    Worker 2 (outro container Python)

Assim, tudo roda com docker-compose up.

Deseja isso?
You said:
se der problema no consumo ficara rodando sem prar
ChatGPT said:

Sim, por padrão, o consumidor (worker) usando pika com start_consuming() vai ficar rodando para sempre, mesmo que:

    A fila esteja vazia ✅

    O RabbitMQ esteja ligado, mas sem mensagens ✅

    Não esteja recebendo mais mensagens por um tempo ✅

🕓 Isso é o comportamento esperado — o worker fica “escutando” a fila continuamente.
🧨 Mas... e se houver algum problema real?

Aqui estão os comportamentos possíveis dependendo do tipo de falha:
❌ 1. RabbitMQ está desligado quando o worker inicia

O que acontece?

    O worker não consegue conectar

    Você recebe uma exceção tipo:

pika.exceptions.AMQPConnectionError

Como lidar:

Você pode usar um try/except + retry, exemplo:

import pika
import time

while True:
    try:
        connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        break  # Conectou? Sai do loop
    except pika.exceptions.AMQPConnectionError:
        print("❌ Não conseguiu conectar. Tentando novamente em 5s...")
        time.sleep(5)

🔌 2. RabbitMQ cai enquanto o worker já está conectado

O que acontece?

    O worker quebra a conexão

    Levanta uma exceção do tipo:

StreamLostError
ConnectionResetError
ChannelClosedByBroker

Como lidar:

Você pode envolver o start_consuming() em um try/except com reconexão automática.
🧪 Exemplo Completo com reconexão automática

import pika
import time

def start_worker():
    while True:
        try:
            connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
            channel = connection.channel()

            channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
            channel.queue_declare(queue='consult_queue')
            channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

            channel.basic_qos(prefetch_count=1)

            def callback(ch, method, properties, body):
                print("📥 Mensagem recebida:", body.decode())
                time.sleep(2)
                ch.basic_ack(delivery_tag=method.delivery_tag)
                print("✅ Processada!")

            channel.basic_consume(queue='consult_queue', on_message_callback=callback)
            print("🟢 Aguardando mensagens...")
            channel.start_consuming()

        except (pika.exceptions.AMQPConnectionError, pika.exceptions.StreamLostError) as e:
            print("❌ Erro de conexão:", str(e))
            print("🔁 Tentando reconectar em 5s...")
            time.sleep(5)
        except KeyboardInterrupt:
            print("🛑 Interrompido pelo usuário.")
            break
        except Exception as e:
            print("🚨 Erro inesperado:", str(e))
            time.sleep(5)

start_worker()

✅ Conclusão

    Sim, o worker fica rodando para sempre, mesmo se não houver mensagens.

    Se houver falha de conexão, o script normal quebra — mas você pode tratar com try/except e um loop.

    Usar esse tipo de reconexão é boa prática, especialmente em produção.

Se quiser, posso montar um worker.py já com tratamento de erro para usar com Docker ou localmente. Deseja isso?
You said:
se houve problema do consumidor como que as mensagens fica persistidas ate serem consumidas de novo
ChatGPT said:

Ótima pergunta! Essa é uma preocupação essencial em sistemas com filas, e o RabbitMQ tem mecanismos específicos de persistência para garantir que mensagens não sejam perdidas, mesmo que o consumidor caia ou o servidor RabbitMQ reinicie.
✅ Resposta direta:

Sim, as mensagens podem ficar persistidas até serem consumidas de novo, desde que você ative os parâmetros corretos:
🧱 1. Fila durável (durable queue)

Isso garante que a fila sobreviva a reinicializações do RabbitMQ.

channel.queue_declare(queue='consult_queue', durable=True)

📩 2. Mensagem persistente

Ao publicar uma mensagem, você precisa indicar que ela é persistente, ou seja, que deve ser salva em disco (não só na memória):

channel.basic_publish(
    exchange='consult_exchange',
    routing_key='links.novo',
    body='Mensagem importante',
    properties=pika.BasicProperties(
        delivery_mode=2  # 2 = mensagem persistente
    )
)

📦 3. Consumer com manual ACK (acknowledgment)

Se o consumidor não der ACK, a mensagem não é removida da fila.
Então, se o consumidor cair no meio do processamento, a mensagem volta para a fila (reentrega automática).

channel.basic_consume(
    queue='consult_queue',
    on_message_callback=callback,
    auto_ack=False  # ✅ IMPORTANTE
)

No callback, você deve enviar o ACK quando terminar o processamento:

ch.basic_ack(delivery_tag=method.delivery_tag)

🔁 O que acontece se o consumidor cair?
Exemplo de fluxo:

    Você publica uma mensagem com delivery_mode=2 (persistente)

    A fila foi declarada como durable=True

    O consumidor começa a processar, mas cai antes do ack

🔁 Resultado:

    O RabbitMQ detecta que a conexão foi perdida

    A mensagem volta para a fila

    Quando o consumidor volta, ela será reentregue

✅ Nada é perdido.
⚠️ ATENÇÃO: Persistência só funciona se tudo estiver configurado
Item	Está persistente?
Fila com durable=True	✅
Mensagem com delivery_mode=2	✅
ACK manual (auto_ack=False)	✅

Se qualquer um desses faltar, as mensagens podem ser perdidas em falhas inesperadas.
💡 Exemplo completo: producer com persistência

channel.queue_declare(queue='consult_queue', durable=True)

channel.basic_publish(
    exchange='consult_exchange',
    routing_key='links.novo',
    body='Pedido 999 criado',
    properties=pika.BasicProperties(
        delivery_mode=2  # Persistente
    )
)

✅ Conclusão

    Sim, o RabbitMQ pode garantir que mensagens não se percam mesmo que o consumidor caia, mas você precisa:

        Declarar a fila como durable=True

        Publicar mensagens com delivery_mode=2

        Usar auto_ack=False e só dar ack quando terminar o processamento

Se quiser, posso montar um exemplo com todos esses pontos prontos, tanto para o producer quanto para o consumer. Deseja isso?
You're now using our basic model.
To access more intelligence, create an account or log in.


ChatGPT can make mistakes. Check important info.


In [ ]:
import pika
import time

def callback(ch, method, properties, body):
    print("🧑‍🔧 [WORKER 2] Recebido:", body.decode())
    time.sleep(2)
    print("✅ [WORKER 2] Processado")
    ch.basic_ack(delivery_tag=method.delivery_tag)

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='consult_queue', on_message_callback=callback, auto_ack=False)

print('🔄 [WORKER 2] Esperando mensagens...')
channel.start_consuming()


In [ ]:
Skip to content
You said:
import pika
url = ['z','e','f']

# Conecta no RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')


for t in url:
    # Envia uma mensagem para pedidos novos
    channel.basic_publish(
        exchange='consult_exchange',
        routing_key='links.novo',
        body='Pedido 123 criado'
    )

# Envia uma mensagem para pedidos cancelados
# channel.basic_publish(
#     exchange='pedidos_ex',
#    routing_key='pedido.cancelado',
#    body='Pedido 456 cancelado'
#)

print("Mensagens enviadas!")
connection.close()

pq nao tem queue
ChatGPT said:

O seu código declara uma exchange (consult_exchange), envia mensagens para ela, mas não declara nenhuma queue nem faz o binding entre uma queue e a exchange. Por isso, as mensagens são perdidas, a menos que já exista alguma fila previamente configurada no RabbitMQ e ligada à exchange.
Por que as mensagens "somem"?

No RabbitMQ, o fluxo básico de mensagens é:

Publisher → Exchange → Queue(s) → Consumer(s)

Se nenhuma queue estiver ligada (bound) à exchange com a routing key correspondente, as mensagens:

    São descartadas automaticamente (não ficam armazenadas em lugar nenhum);

    E não causam erro no basic_publish() – ou seja, o seu código continua funcionando "normalmente", mesmo sem fazer nada útil.

Como corrigir

Você precisa:

    Declarar uma fila (queue).

    Fazer o binding dessa fila com a exchange usando a mesma routing_key que você usa para publicar.

Código corrigido com fila

import pika

url = ['z', 'e', 'f']

# Conecta ao RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')

# Declara a fila
channel.queue_declare(queue='consult_queue')

# Faz o binding da fila à exchange com a routing key usada
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Envia mensagens
for t in url:
    channel.basic_publish(
        exchange='consult_exchange',
        routing_key='links.novo',
        body=f'Pedido 123 criado: {t}'
    )

print("Mensagens enviadas!")
connection.close()

Dica extra: Debug – como saber se a fila recebeu?

Você pode usar o painel de administração do RabbitMQ (geralmente acessível em http://localhost:15672) para ver:

    Se a queue foi criada;

    Se ela tem mensagens prontas para serem consumidas;

    Se a exchange e a queue estão ligadas corretamente.

Se quiser, posso te ajudar também a escrever o código do consumer que consome da fila consult_queue. Deseja isso?
You said:
exchange, queue,  routing_key como que funcionam
ChatGPT said:

Ótima pergunta! Entender bem como funcionam exchange, queue e routing_key é essencial para usar o RabbitMQ corretamente.

Vamos explicar de forma simples e objetiva, com analogias e exemplos práticos.
🧠 Conceitos Básicos
✅ exchange (troca, distribuidor de mensagens)

A exchange é responsável por receber mensagens de um produtor (publisher) e decidir para onde elas vão.

Ela não armazena mensagens – só as encaminha para as filas (queues).
✅ queue (fila)

A fila é onde a mensagem fica armazenada até ser consumida por um consumidor (consumer).

    Se ninguém estiver consumindo a mensagem, ela fica "parada" na fila esperando.

✅ routing_key (chave de roteamento)

É uma etiqueta que acompanha a mensagem e ajuda a exchange a decidir para qual fila a mensagem vai (baseado nas regras de binding entre exchange e fila).
🧭 Como tudo se conecta?

Aqui está um resumo visual e conceitual:

[ Producer ]
     |
     |  (routing_key='user.created')
     v
[ Exchange ]  --------> [ Queue A ]
                 \
                  ---> [ Queue B ]

🧪 Tipos de Exchange

Existem 4 tipos principais de exchange no RabbitMQ:
Tipo de Exchange	O que faz	routing_key usada?	Exemplo
direct	Entrega para filas com routing_key exata	✅ Sim	Mensagem com key user.created vai para fila ligada com essa key
fanout	Entrega para todas as filas ligadas	❌ Ignorada	Broadcast (ex: enviar mensagem para todos)
topic	Entrega com base em padrões (wildcards)	✅ Sim + curingas	user.*, user.#
headers	Usa headers em vez de routing_key	❌ Usa headers	Avançado – raramente usado
📦 Exemplo prático com direct

Imagine um sistema de pedidos:

    Temos uma exchange chamada pedidos_ex

    Temos duas filas:

        fila_novos_pedidos

        fila_pedidos_cancelados

Fazemos o binding assim:

channel.queue_bind(exchange='pedidos_ex', queue='fila_novos_pedidos', routing_key='pedido.novo')
channel.queue_bind(exchange='pedidos_ex', queue='fila_pedidos_cancelados', routing_key='pedido.cancelado')

Agora, se você publicar:

channel.basic_publish(exchange='pedidos_ex', routing_key='pedido.novo', body='Pedido 123 criado')

➡️ A mensagem vai para fila_novos_pedidos.
🧠 Resumo rápido
Elemento	Responsável por...
exchange	Receber a mensagem e decidir para onde ela vai
queue	Guardar a mensagem até alguém consumir
routing_key	Definir como a mensagem é roteada para as filas

Se quiser, posso te mostrar um exemplo completo com produtor + consumidor, ou um gráfico explicando visualmente com setas. Deseja isso?
You said:
etiqueta ajuda? como assim
ChatGPT said:

Ótima observação. Vamos esclarecer isso.

Quando eu disse que a routing_key é como uma etiqueta, quis dizer o seguinte:
🎯 Analogia: Correios

Imagine que você está enviando uma carta.

    Você escreve o endereço do destinatário no envelope.

    O carteiro (🐰 exchange) lê o endereço e entrega a carta para a pessoa certa (fila).

👉 Neste caso:

    A carta é a mensagem (body)

    O endereço é a routing_key

    O carteiro é a exchange

    A casa do destinatário é a queue

Se o endereço estiver errado ou ninguém tiver uma casa com aquele nome/endereço, a carta não será entregue.
🧪 No RabbitMQ

A routing_key funciona da mesma forma:
é uma etiqueta (chave de roteamento) usada para decidir para qual fila a mensagem deve ir.
Exemplo prático:

Vamos dizer que temos essas filas ligadas a uma exchange do tipo direct:

channel.queue_bind(exchange='pedidos_ex', queue='fila_novos', routing_key='pedido.novo')
channel.queue_bind(exchange='pedidos_ex', queue='fila_cancelados', routing_key='pedido.cancelado')

Agora, se você envia:

channel.basic_publish(exchange='pedidos_ex', routing_key='pedido.novo', body='Novo pedido!')

A exchange vê a routing_key = 'pedido.novo' e pensa:

    “Ah! A fila fila_novos quer mensagens com essa etiqueta, vou mandar pra ela.”

Mas se você usar uma chave diferente, tipo:

routing_key = 'pedido.reembolsado'

E nenhuma fila estiver ligada com essa chave, a exchange vai descartar a mensagem.
✅ Moral da história:

A routing_key é uma forma de etiquetar a mensagem para que a exchange saiba para onde mandar.

Se ninguém estiver esperando por essa etiqueta (chave), a mensagem se perde.

Se quiser, posso montar um pequeno exemplo completo (com código) onde fica bem fácil visualizar tudo isso. Quer?
You said:
import pika

url = ['z', 'e', 'f']

# Conecta ao RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')

# Declara a fila
channel.queue_declare(queue='consult_queue')

# Faz o binding da fila à exchange com a routing key usada
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Envia mensagens
for t in url:
    print(t)
    channel.basic_publish(
        exchange='consult_exchange',
        routing_key='links.novo',
        body=f'Pedido 123 criado: {t}'
    )

print("Mensagens enviadas!")
connection.close()



import pika
import time

def callback(ch, method, properties, body):
    print("📥 [NOVO] Recebido:", body.decode())

    # Simula um processamento
    time.sleep(2)

    print("✅ [NOVO] Processado com sucesso.")
    
    # Envia ack manual
    ch.basic_ack(delivery_tag=method.delivery_tag)

# Conexão
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange e a fila
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')

# Liga a fila à exchange
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Limita a 1 mensagem por consumidor
channel.basic_qos(prefetch_count=1)

# Escuta a fila com ack manual
channel.basic_consume(
    queue='fila_novos_pedidos',
    on_message_callback=callback,
    auto_ack=False  # <- importante!
)

print('🟢 Aguardando novos pedidos...')
channel.start_consuming()
ChatGPT said:

Ótimo que você já montou o producer e o consumer com o RabbitMQ! 🚀
Mas tem um erro discreto no seu código que vai impedir o consumer de funcionar corretamente. Vamos revisar e corrigir juntos.
🧨 Problema principal

No seu consumer, você está escutando uma fila chamada:

queue='fila_novos_pedidos'

Porém, no producer, você está declarando e enviando mensagens para a fila chamada:

queue='consult_queue'

👉 Ou seja, o consumer está esperando em uma fila diferente daquela que o producer está usando.
✅ Solução

Troque o nome da fila no consumer para consult_queue, igual ao do producer.
✅ Código corrigido (consumer):

import pika
import time

def callback(ch, method, properties, body):
    print("📥 [NOVO] Recebido:", body.decode())

    # Simula um processamento
    time.sleep(2)

    print("✅ [NOVO] Processado com sucesso.")
    
    # Envia ack manual
    ch.basic_ack(delivery_tag=method.delivery_tag)

# Conexão
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange e a fila
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')

# Liga a fila à exchange
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Limita a 1 mensagem por consumidor
channel.basic_qos(prefetch_count=1)

# Escuta a fila com ack manual
channel.basic_consume(
    queue='consult_queue',  # CORRIGIDO AQUI!
    on_message_callback=callback,
    auto_ack=False
)

print('🟢 Aguardando novos pedidos...')
channel.start_consuming()

🧪 Agora o fluxo vai funcionar assim:

    O producer envia 3 mensagens com routing_key='links.novo' para a exchange='consult_exchange'.

    A exchange entrega as mensagens para a queue='consult_queue', que está ligada corretamente.

    O consumer consome essas mensagens, uma por vez, e imprime cada uma delas.

Se quiser, posso te ajudar a testar isso com múltiplos consumidores ou simular falhas. Está funcionando agora?
You said:
How to get message
ChatGPT said:

To get messages manually from the queue using the RabbitMQ Management Web UI, follow these steps based on your screenshot:
✅ How to Get (Read) Messages from the Queue
1. Go to the Queue Page

You're already there — the queue name is consult_queue.
2. Scroll Down to the “Get messages” Section

You will see this block (you already highlighted it):

    Get messages
    Warning: getting messages from a queue is a destructive action.
    [...]

3. Configure the Get Settings

Use the following values:

    Ack Mode:
    ✔ Ack message requeue false (or true if you want to leave the message in the queue)

    Encoding:
    ✔ Auto string / base64

    Messages:
    ✔ 1 (or however many messages you want to read)

4. Click “Get Message(s)”

After clicking it, a section just below will appear showing the message(s) retrieved, something like:

Payload: Pedido 123 criado: z

Or similar, depending on what you published.
📌 Notes

    This is a manual fetch, not consumption by a consumer.

    Once retrieved (with requeue false), the message is removed from the queue.

    If the queue is empty, it will simply say No messages.

🛠 Tip: Use It Only for Debug/Test

This tool is super useful for debugging or inspecting a few messages, but in production, messages should always be consumed by a consumer script (like the one you wrote with start_consuming()).

Would you like to test requeueing or simulate multiple pending messages too?
You said:
how it  would be
ChatGPT said:

Great — let's walk through how to use the “Get Message(s)” feature with different options, including requeueing, fetching multiple messages, and what each setting means in practice.
🔄 1. Requeue = true vs false
Option	What Happens
Ack message requeue false	Message is removed from the queue permanently
Ack message requeue true	Message is kept in the queue for other consumers
✅ When to use each:

    Use false if you're just trying to consume the message.

    Use true if you want to peek at the message without removing it.

📦 2. Number of Messages

You can choose how many messages to pull from the queue at once.

    Messages: 1 → Gets 1 message (default)

    Messages: 10 → Gets 10 messages (if available)

    ⚠️ These messages will all be removed if requeue = false.

🛠 Example Scenarios
✅ Scenario 1: Consume 1 message (like a real consumer)

    Ack Mode: Ack message requeue false

    Messages: 1

    ✅ This simulates a consumer pulling 1 message and removing it from the queue.

👁️ Scenario 2: Peek at message, leave in queue

    Ack Mode: Ack message requeue true

    Messages: 1

    ✅ Lets you see the message, but not delete it — good for debugging.

🧪 Scenario 3: Drain all messages for inspection

    Ack Mode: Ack message requeue false

    Messages: 10 (or higher if you know more messages exist)

    ✅ Use this to quickly clear the queue manually and inspect contents.

🧼 How to Clear a Queue Completely

If you want to clear all messages from a queue (not just get them):

    Go to the queue (consult_queue)

    Click on the "Purge" button (top-right area)

    Confirm

⚠️ This removes all messages immediately, with no chance to recover.
🔁 Want to simulate messages to test?

You can use the “Publish Message” section above to send test messages:

    Exchange: consult_exchange

    Routing key: links.novo

    Payload: Test message

Then use “Get Message(s)” to read it.

Would you like an example of publishing and retrieving a message step-by-step using the UI?
You said:
me ajuda a completar e conseguir get message
ChatGPT said:

Claro! Vamos passo a passo para que você consiga:

    Publicar uma mensagem corretamente na fila (consult_queue)

    E depois usar o botão “Get Message(s)” para recuperar essa mensagem

✅ ETAPA 1: Publicar uma mensagem corretamente

Siga isso na interface do RabbitMQ (Web UI):
Vá até a aba da fila consult_queue.
Role até a seção “Publish message” e preencha assim:

    Exchange:
    consult_exchange

    Routing key:
    links.novo

        Isso é essencial — tem que ser exatamente igual ao binding da fila.

    Payload:
    Coloque qualquer coisa, exemplo:
    Pedido de teste 001

    Headers e Properties:
    Pode deixar em branco.

    Delivery mode:
    Pode deixar como está (1 - Non-persistent)

✅ Clique em “Publish message”

Se tudo estiver certo, a mensagem será roteada pela exchange para a fila consult_queue.
✅ ETAPA 2: Recuperar (get) a mensagem da fila
Logo abaixo, vá para a seção “Get messages” e configure:

    Ack mode:
    Ack message requeue false ✅
    (isso consome a mensagem — ela será removida da fila)

    Encoding:
    Auto string / base64

    Messages:
    1

✅ Clique em “Get Message(s)”
🟢 Resultado Esperado:

    Um bloco com o conteúdo da mensagem vai aparecer abaixo.

    Exemplo de resposta:

Payload: Pedido de teste 001

Se aparecer isso: Queue is empty, pode ser que:

    A mensagem não foi enviada corretamente;

    A routing_key estava errada;

    Ou alguém (um consumer?) já consumiu a mensagem.

🧪 Dica de Teste Rápido

Repita o processo:

    Publica a mensagem novamente

    Em menos de 5 segundos, vá no “Get Message(s)” e clique

Se ainda assim não funcionar, me diga:

    O que você colocou nos campos da publicação

    O que aparece no resultado do Get

Posso te ajudar corrigindo em tempo real.


ChatGPT can make mistakes. Check important info.


In [ ]:
import pika

url = ['z', 'e', 'f']

# Conecta ao RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')

# Declara a fila
channel.queue_declare(queue='consult_queue')

# Faz o binding da fila à exchange com a routing key usada
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Envia mensagens
for t in url:
    channel.basic_publish(
        exchange='consult_exchange',
        routing_key='links.novo',
        body=f'Pedido 123 criado: {t}'
    )

print("Mensagens enviadas!")
connection.close()


In [3]:
import pika
import time

def callback(ch, method, properties, body):
    print("📥 [NOVO] Recebido:", body.decode())

    # Simula um processamento
    time.sleep(2)

    print("✅ [NOVO] Processado com sucesso.")
    
    # Envia ack manual
    ch.basic_ack(delivery_tag=method.delivery_tag)

# Conexão
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declara a exchange e a fila
channel.exchange_declare(exchange='consult_exchange', exchange_type='direct')
channel.queue_declare(queue='consult_queue')

# Liga a fila à exchange
channel.queue_bind(exchange='consult_exchange', queue='consult_queue', routing_key='links.novo')

# Limita a 1 mensagem por consumidor
channel.basic_qos(prefetch_count=1)

# Escuta a fila com ack manual
channel.basic_consume(
    queue='consult_queue',  # CORRIGIDO AQUI!
    on_message_callback=callback,
    auto_ack=False
)

print('🟢 Aguardando novos pedidos...')
channel.start_consuming()


🟢 Aguardando novos pedidos...
📥 [NOVO] Recebido: Pedido 123 criado: z
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: e
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: f
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: z
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: e
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: f
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: 
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: z
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: e
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: f
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: z
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: e
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: Pedido 123 criado: f
✅ [NOVO] Processado com sucesso.
📥 [NOVO] Recebido: 
✅ [NOVO] Processado c

KeyboardInterrupt: 